# Lab 2b: Deep Learning Model (Part 1): Twitter Sentiment Analysis - Data cleaning and Data exploration
by Group 8: Lu Han, Tony Chan, Michaela Hrabetova, Sangeeta Khanna, Cristina Endara

# Original Source

Work is based on "Another Twitter sentiment analysis with Python" by Ricky Kim on [towardsdatascience.com](https://towardsdatascience.com/another-twitter-sentiment-analysis-bb5b01ebad90)

There are heavy modifications to the original work for code enhancements, encapsulation, improvement, bug fixing in order to make the original source work for this project. Eventually, a realization of deep learning model within a python class called SentimentInferencing. Python source can be found in sentiment.py

In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
cols = ['sentiment','id','date','query_string','user','text']
df = pd.read_csv("../input/training.1600000.processed.noemoticon.csv",header=None, names=cols, encoding='latin-1')
# above line will be different depending on where you saved your data, and your file name
df.head()

,sentiment,id,date,query_string,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [2]:
df.sentiment.value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

In [3]:
df.drop(['id','date','query_string','user'],axis=1,inplace=True)

In [4]:
# Negative Sentiment
df[df.sentiment == 0].sample(5)

,sentiment,text
696140,0,Who does a car wash in the rain?! I guess we do
417505,0,"Greg is going to bed, and i'm not ready to go ..."
534016,0,@MEazyVA omg I don't have that!! now I won't ...
448647,0,@AmberLovesNKOTB I didn't even touch danny and...
648294,0,Or maybe not - the one thing it lacks is the a...


In [5]:
# Positive Sentiment
df[df.sentiment == 4].sample(5)

,sentiment,text
1458917,4,@brendanchng dun beee! nothing an ice cream ca...
980908,4,@mattycus Yes.
1521193,4,@fawksbeaumont I can't draw at work.
1276224,4,is bored with cruel intentions and is now watc...
1252465,4,On this gay site.....


In [6]:
df['pre_clean_len'] = [len(t) for t in df.text]

In [7]:
from pprint import pprint
data_dict = {
    'sentiment':{
        'type':df.sentiment.dtype,
        'description':'sentiment class - 0:negative, 1:positive'
    },
    'text':{
        'type':df.text.dtype,
        'description':'tweet text'
    },
    'pre_clean_len':{
        'type':df.pre_clean_len.dtype,
        'description':'Length of the tweet before cleaning'
    },
    'dataset_shape':df.shape
}
pprint(data_dict)

{'dataset_shape': (1600000, 3),
 'pre_clean_len': {'description': 'Length of the tweet before cleaning',
                   'type': dtype('int64')},
 'sentiment': {'description': 'sentiment class - 0:negative, 1:positive',
               'type': dtype('int64')},
 'text': {'description': 'tweet text', 'type': dtype('O')}}


In [8]:
df.text[279]

"Whinging. My client&amp;boss don't understand English well. Rewrote some text unreadable. It's written by v. good writer&amp;reviewed correctly. "

In [9]:
from bs4 import BeautifulSoup
example1 = BeautifulSoup(df.text[279], 'lxml')
print(example1.get_text())

Whinging. My client&boss don't understand English well. Rewrote some text unreadable. It's written by v. good writer&reviewed correctly. 


In [10]:
df.text[343]

'@TheLeagueSF Not Fun &amp; Furious? The new mantra for the Bay 2 Breakers? It was getting 2 rambunctious;the city overreacted &amp; clamped down '

In [11]:
import re
re.sub(r'@[A-Za-z0-9]+','',df.text[343])

' Not Fun &amp; Furious? The new mantra for the Bay 2 Breakers? It was getting 2 rambunctious;the city overreacted &amp; clamped down '

In [12]:
df.text[0]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

In [13]:
re.sub('https?://[A-Za-z0-9./]+','',df.text[0])

"@switchfoot  - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

In [14]:
df.text[226]

'Tuesdayï¿½ll start with reflection ï¿½n then a lecture in Stress reducing techniques. That sure might become very useful for us accompaniers '

In [15]:
# this part is not working...
# testing = df.text[226].decode("utf-8-sig")
testing = df.text[226]
testing

'Tuesdayï¿½ll start with reflection ï¿½n then a lecture in Stress reducing techniques. That sure might become very useful for us accompaniers '

In [16]:
testing.replace(u"\ufffd", "?")

'Tuesdayï¿½ll start with reflection ï¿½n then a lecture in Stress reducing techniques. That sure might become very useful for us accompaniers '

In [17]:
df.text[175]

"@machineplay I'm so sorry you're having to go through this. Again.  #therapyfail"

In [18]:
re.sub("[^a-zA-Z]", " ", df.text[175])

' machineplay I m so sorry you re having to go through this  Again    therapyfail'

In [19]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

His code for applying tweet_cleaner_updated is not python/pandas way, therefore it is slow

In [20]:
# nums = [0,400000,800000,1200000,1600000]
# print("Cleaning and parsing the tweets...\n")
# clean_tweet_texts = []
# for i in range(nums[0],nums[1]):
#     if( (i+1)%10000 == 0 ):
#         print("Tweets %d of %d has been processed" % (i+1, nums[1]))                                                                    
#     clean_tweet_texts.append(tweet_cleaner_updated(df['text'][i]))

In [21]:
# clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
# clean_df['target'] = df.sentiment
# clean_df.head()

In [22]:
df['filtered_text'] = df['text'].apply(tweet_cleaner_updated)
clean_df = df[['sentiment','filtered_text']]
clean_df = clean_df.rename(columns={'sentiment':'target','filtered_text':'text'})
clean_df.head(5)

,target,text
0,0,awww that bummer you shoulda got david carr of...
1,0,is upset that he can not update his facebook b...
2,0,dived many times for the ball managed to save ...
3,0,my whole body feels itchy and like its on fire
4,0,no it not behaving at all mad why am here beca...


In [23]:
# clean_df = test_df[['sentiment','filtered_text']]
# clean_df

In [24]:
# clean_df = clean_df.rename(columns={'sentiment':'target','filtered_text':'text'})
# clean_df.head()

In [25]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   text    1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [26]:
clean_df.isnull().sum()

target    0
text      0
dtype: int64

In [27]:
# clean_df.to_csv('../data/clean_tweet.csv',encoding='utf-8')
my_df = pd.read_csv('../data/clean_tweet.csv',index_col=0)
my_df.head()

,target,text
0,0,awww that bummer you shoulda got david carr of...
1,0,is upset that he can not update his facebook b...
2,0,dived many times for the ball managed to save ...
3,0,my whole body feels itchy and like its on fire
4,0,no it not behaving at all mad why am here beca...
